# Intro to Machine Learning Jupyter Notebook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
file_path = "StormEvents_details-ftp_v1.0_d2024_c20250401.csv.gz" # data

df = pd.read_csv(file_path)

df = df[pd.notna(df['DAMAGE_PROPERTY'])] # keep only rows that are not empty in property damage

df = df.drop(columns=['EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'])

display(df.head(5))



,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,BEGIN_RANGE,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
1,202411,16,230,202411,18,1421,197838,1223377,OREGON,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202405,19,1839,202405,19,1902,190905,1184919,OKLAHOMA,40,...,8.0,WNW,CUSTER CITY,5.0,N,CUSTER CITY,35.7100,-99.0010,35.7370,-98.8910
3,202405,23,2155,202405,23,2155,190907,1180805,OKLAHOMA,40,...,2.0,W,NINNEKAH,2.0,W,NINNEKAH,34.9501,-97.9523,34.9501,-97.9523
4,202405,24,1405,202405,24,1410,191916,1182348,MISSISSIPPI,28,...,0.0,N,ALGOMA,0.0,N,ALGOMA,34.1800,-89.0300,34.1800,-89.0300
5,202411,1,0,202411,1,1600,197531,1221908,WASHINGTON,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Engineering


# Clustering

# Most association with different features